# ACHTUNG: lange Laufzeit - Ausgaben nicht unüberlegt löschen

zur Not die "clean newsgroups" wieder einlesen - spart immerhin die Normalisierung

In [ ]:
!spacy download en_core_web_sm

In [1]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import text_normalizer as tn
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


C:\Users\yhutter\GitRepos\fhgr-ta\code\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\yhutter\GitRepos\fhgr-ta\code\venv\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
C:\Users\yhutter\GitRepos\fhgr-ta\code\venv\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
C:\Users\yhutter\AppData\Local\Temp\ipykernel_13756\1315687842.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to all

In [2]:
#daten herunterladen/einladen
data = fetch_20newsgroups(subset='all', shuffle=True,
                          remove=('headers', 'footers', 'quotes'))
data_labels_map = dict(enumerate(data.target_names))

In [3]:
#daten in DF
corpus, target_labels, target_names = (data.data, data.target, 
                                       [data_labels_map[label] for label in data.target])
data_df = pd.DataFrame({'Article': corpus, 'Target Label': target_labels, 'Target Name': target_names})
print(data_df.shape)
data_df.head(10)

(18846, 3)


,Article,Target Label,Target Name
0,\n\nI am sure some bashers of Pens fans are pr...,10,rec.sport.hockey
1,My brother is in the market for a high-perform...,3,comp.sys.ibm.pc.hardware
2,\n\n\n\n\tFinally you said what you dream abou...,17,talk.politics.mideast
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,3,comp.sys.ibm.pc.hardware
4,1) I have an old Jasmine drive which I cann...,4,comp.sys.mac.hardware
5,\n\nBack in high school I worked as a lab assi...,12,sci.electronics
6,\n\nAE is in Dallas...try 214/241-6060 or 214/...,4,comp.sys.mac.hardware
7,"\n[stuff deleted]\n\nOk, here's the solution t...",10,rec.sport.hockey
8,"\n\n\nYeah, it's the second one. And I believ...",10,rec.sport.hockey
9,\nIf a Christian means someone who believes in...,19,talk.religion.misc


In [4]:
#Datensatz ist unbereinigt
total_nulls = data_df[data_df.Article.str.strip() == ''].shape[0]
print("Empty documents:", total_nulls)

Empty documents: 515


In [5]:
#leere Daten löschen
data_df = data_df[~(data_df.Article.str.strip() == '')]
data_df.shape

(18331, 3)

Achtung: Ausführung Cleaning dauert durch die Lemmatisierung recht lange - anderes Modell nehmen?

In [ ]:
#das bekannte cleaning
import nltk
stopword_list = nltk.corpus.stopwords.words('english')
# just to keep negation if any in bi-grams
stopword_list.remove('no')
stopword_list.remove('not')

# normalize our corpus
norm_corpus = tn.normalize_corpus(corpus=data_df['Article'], html_stripping=True, contraction_expansion=True, 
                                  accented_char_removal=True, text_lower_case=True, text_lemmatization=True, 
                                  text_stemming=False, special_char_removal=True, remove_digits=True,
                                  stopword_removal=True, stopwords=stopword_list)
data_df['Clean Article'] = norm_corpus

In [17]:
#Übersicht
data_df = data_df[['Article', 'Clean Article', 'Target Label', 'Target Name']]
data_df.head(10)

,Article,Clean Article,Target Label,Target Name
0,\n\nI am sure some bashers of Pens fans are pr...,sure basher pen fan pretty confused lack kind ...,10,rec.sport.hockey
1,My brother is in the market for a high-perform...,brother market high performance video card sup...,3,comp.sys.ibm.pc.hardware
2,\n\n\n\n\tFinally you said what you dream abou...,finally say dream mediterranean new area great...,17,talk.politics.mideast
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,think scsi card dma transfer not disk scsi car...,3,comp.sys.ibm.pc.hardware
4,1) I have an old Jasmine drive which I cann...,old jasmine drive not use new system understan...,4,comp.sys.mac.hardware
5,\n\nBack in high school I worked as a lab assi...,back high school work lab assistant bunch expe...,12,sci.electronics
6,\n\nAE is in Dallas...try 214/241-6060 or 214/...,ae dallas try tech support may line one get start,4,comp.sys.mac.hardware
7,"\n[stuff deleted]\n\nOk, here's the solution t...",stuff delete ok solution problem move canada y...,10,rec.sport.hockey
8,"\n\n\nYeah, it's the second one. And I believ...",yeah second one believe price try get good loo...,10,rec.sport.hockey
9,\nIf a Christian means someone who believes in...,christian mean someone believe divinity jesus ...,19,talk.religion.misc


In [18]:
#durch das cleaning leere Dokumente? (Leerzeichen ersetzen)
data_df = data_df.replace(r'^(\s?)+$', np.nan, regex=True)
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18846 entries, 0 to 18845
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Article        18331 non-null  object
 1   Clean Article  18301 non-null  object
 2   Target Label   18846 non-null  int64 
 3   Target Name    18846 non-null  object
dtypes: int64(1), object(3)
memory usage: 589.1+ KB


In [19]:
#löschen
data_df = data_df.dropna().reset_index(drop=True)
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18301 entries, 0 to 18300
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Article        18301 non-null  object
 1   Clean Article  18301 non-null  object
 2   Target Label   18301 non-null  int64 
 3   Target Name    18301 non-null  object
dtypes: int64(1), object(3)
memory usage: 572.0+ KB


In [20]:
#HR: Pfad angepasst - das muss unbedingt geändert werden!

data_df.to_csv('clean_newsgroups.csv', index=False)

In [8]:
#HR: Pfad angepasst - s.o.

data_df = pd.read_csv('clean_newsgroups.csv')

In [9]:
#Dokumente aufteilen 33% testdaten
from sklearn.model_selection import train_test_split

train_corpus, test_corpus, train_label_nums, test_label_nums, train_label_names, test_label_names =\
                                 train_test_split(np.array(data_df['Clean Article']), np.array(data_df['Target Label']),
                                                       np.array(data_df['Target Name']), test_size=0.33, random_state=42)

train_corpus.shape, test_corpus.shape

((12261,), (6040,))

In [10]:
#wie verteilen sich die Dokumente nach Newsgruppen?
from collections import Counter

trd = dict(Counter(train_label_names))
tsd = dict(Counter(test_label_names))

(pd.DataFrame([[key, trd[key], tsd[key]] for key in trd], 
             columns=['Target Label', 'Train Count', 'Test Count'])
.sort_values(by=['Train Count', 'Test Count'],
             ascending=False))

,Target Label,Train Count,Test Count
6,sci.crypt,675,287
8,comp.windows.x,669,311
1,comp.graphics,661,292
5,rec.motorcycles,658,311
2,soc.religion.christian,656,318
0,rec.sport.hockey,655,318
14,sci.electronics,645,311
15,comp.os.ms-windows.misc,642,304
18,sci.med,641,319
7,rec.sport.baseball,641,310


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score

# build BOW features on train articles
cv = CountVectorizer(binary=False, min_df=0.0, max_df=1.0)
cv_train_features = cv.fit_transform(train_corpus)

In [12]:
# transform test articles into features
# HR: der "fit" wird übernommen
cv_test_features = cv.transform(test_corpus)

In [13]:
print('BOW model:> Train features shape:', cv_train_features.shape, ' Test features shape:', cv_test_features.shape)

BOW model:> Train features shape: (12261, 74077)  Test features shape: (6040, 74077)


In [14]:
#HR erste Runde Algorithmen: basierend auf BoW (hier: cv)

from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB(alpha=1)
mnb.fit(cv_train_features, train_label_names)
mnb_bow_cv_scores = cross_val_score(mnb, cv_train_features, train_label_names, cv=5)
mnb_bow_cv_mean_score = np.mean(mnb_bow_cv_scores)
print('CV Accuracy (5-fold):', mnb_bow_cv_scores)
print('Mean CV Accuracy:', mnb_bow_cv_mean_score)
mnb_bow_test_score = mnb.score(cv_test_features, test_label_names)
print('Test Accuracy:', mnb_bow_test_score) #auf den Testwerten

CV Accuracy (5-fold): [0.68324501 0.64763458 0.6774062  0.66884176 0.6680261 ]
Mean CV Accuracy: 0.6690307304236447
Test Accuracy: 0.6819536423841059


In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty='l2', max_iter=100, C=1, random_state=42)
lr.fit(cv_train_features, train_label_names)
lr_bow_cv_scores = cross_val_score(lr, cv_train_features, train_label_names, cv=5)
lr_bow_cv_mean_score = np.mean(lr_bow_cv_scores)
print('CV Accuracy (5-fold):', lr_bow_cv_scores)
print('Mean CV Accuracy:', lr_bow_cv_mean_score)
lr_bow_test_score = lr.score(cv_test_features, test_label_names)
print('Test Accuracy:', lr_bow_test_score)

In [46]:
from sklearn.svm import LinearSVC

svm = LinearSVC(penalty='l2', C=1, random_state=42)
svm.fit(cv_train_features, train_label_names)
svm_bow_cv_scores = cross_val_score(svm, cv_train_features, train_label_names, cv=5)
svm_bow_cv_mean_score = np.mean(svm_bow_cv_scores)
print('CV Accuracy (5-fold):', svm_bow_cv_scores)
print('Mean CV Accuracy:', svm_bow_cv_mean_score)
svm_bow_test_score = svm.score(cv_test_features, test_label_names)
print('Test Accuracy:', svm_bow_test_score)

CV Accuracy (5-fold): [0.62821035 0.63417618 0.65334421 0.64559543 0.63907015]
Mean CV Accuracy: 0.6400792650607938
Test Accuracy: 0.6569536423841059


In [44]:
# SVM with Stochastic Gradient Descent
from sklearn.linear_model import SGDClassifier

svm_sgd = SGDClassifier(loss='hinge', penalty='l2', max_iter=5, random_state=42)
svm_sgd.fit(cv_train_features, train_label_names)
svmsgd_bow_cv_scores = cross_val_score(svm_sgd, cv_train_features, train_label_names, cv=5)
svmsgd_bow_cv_mean_score = np.mean(svmsgd_bow_cv_scores)
print('CV Accuracy (5-fold):', svmsgd_bow_cv_scores)
print('Mean CV Accuracy:', svmsgd_bow_cv_mean_score)
svmsgd_bow_test_score = svm_sgd.score(cv_test_features, test_label_names)
print('Test Accuracy:', svmsgd_bow_test_score)

CV Accuracy (5-fold): [0.64166327 0.62683524 0.64641109 0.65171289 0.62724307]
Mean CV Accuracy: 0.6387731106631757
Test Accuracy: 0.6519867549668874


In [30]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=10, random_state=42)
rfc.fit(cv_train_features, train_label_names)
rfc_bow_cv_scores = cross_val_score(rfc, cv_train_features, train_label_names, cv=5)
rfc_bow_cv_mean_score = np.mean(rfc_bow_cv_scores)
print('CV Accuracy (5-fold):', rfc_bow_cv_scores)
print('Mean CV Accuracy:', rfc_bow_cv_mean_score)
rfc_bow_test_score = rfc.score(cv_test_features, test_label_names)
print('Test Accuracy:', rfc_bow_test_score)

CV Accuracy (5-fold): [0.51977171 0.50530179 0.52610114 0.5134584  0.51101142]
Mean CV Accuracy: 0.5151288930091262
Test Accuracy: 0.5268211920529802


In [31]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(n_estimators=10, random_state=42)
gbc.fit(cv_train_features, train_label_names)
gbc_bow_cv_scores = cross_val_score(gbc, cv_train_features, train_label_names, cv=5)
gbc_bow_cv_mean_score = np.mean(gbc_bow_cv_scores)
print('CV Accuracy (5-fold):', gbc_bow_cv_scores)
print('Mean CV Accuracy:', gbc_bow_cv_mean_score)
gbc_bow_test_score = gbc.score(cv_test_features, test_label_names)
print('Test Accuracy:', gbc_bow_test_score)

CV Accuracy (5-fold): [0.54790053 0.53017945 0.55831974 0.56076672 0.55954323]
Mean CV Accuracy: 0.5513419330725968
Test Accuracy: 0.5551324503311258


In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

# build BOW features on train articles
tv = TfidfVectorizer(use_idf=True, min_df=0.0, max_df=1.0)
tv_train_features = tv.fit_transform(train_corpus)

In [33]:
# transform test articles into features
tv_test_features = tv.transform(test_corpus)

In [34]:
print('TFIDF model:> Train features shape:', tv_train_features.shape, ' Test features shape:', tv_test_features.shape)

TFIDF model:> Train features shape: (12261, 74077)  Test features shape: (6040, 74077)


In [35]:
#HR: zweite Runde Klassifikation, basierend auf tf-idf
# Idee: tf-idf liefert bessere Resultate, da Wörter anders gewichtet (nicht nur Anzahl)

mnb = MultinomialNB(alpha=1)
mnb.fit(tv_train_features, train_label_names)
mnb_tfidf_cv_scores = cross_val_score(mnb, tv_train_features, train_label_names, cv=5)
mnb_tfidf_cv_mean_score = np.mean(mnb_tfidf_cv_scores)
print('CV Accuracy (5-fold):', mnb_tfidf_cv_scores)
print('Mean CV Accuracy:', mnb_tfidf_cv_mean_score)
mnb_tfidf_test_score = mnb.score(tv_test_features, test_label_names)
print('Test Accuracy:', mnb_tfidf_test_score)

CV Accuracy (5-fold): [0.71300448 0.68637847 0.71574225 0.71655791 0.71044046]
Mean CV Accuracy: 0.708424714152993
Test Accuracy: 0.7100993377483443


In [36]:
lr = LogisticRegression(penalty='l2', max_iter=100, C=1, random_state=42)
lr.fit(tv_train_features, train_label_names)
lr_tfidf_cv_scores = cross_val_score(lr, tv_train_features, train_label_names, cv=5)
lr_tfidf_cv_mean_score = np.mean(lr_tfidf_cv_scores)
print('CV Accuracy (5-fold):', lr_tfidf_cv_scores)
print('Mean CV Accuracy:', lr_tfidf_cv_mean_score)
lr_tfidf_test_score = lr.score(tv_test_features, test_label_names)
print('Test Accuracy:', lr_tfidf_test_score)

CV Accuracy (5-fold): [0.74317163 0.71818923 0.75326264 0.75081566 0.73613377]
Mean CV Accuracy: 0.740314586327359
Test Accuracy: 0.7521523178807947


In [39]:
svm = LinearSVC(penalty='l2', C=1, random_state=42)
svm.fit(tv_train_features, train_label_names)
svm_tfidf_cv_scores = cross_val_score(svm, tv_train_features, train_label_names, cv=5)
svm_tfidf_cv_mean_score = np.mean(svm_tfidf_cv_scores)
print('CV Accuracy (5-fold):', svm_tfidf_cv_scores)
print('Mean CV Accuracy:', svm_tfidf_cv_mean_score)
svm_tfidf_test_score = svm.score(tv_test_features, test_label_names)
print('Test Accuracy:', svm_tfidf_test_score)

CV Accuracy (5-fold): [0.75295556 0.73694943 0.76305057 0.77079935 0.74632953]
Mean CV Accuracy: 0.7540168878006025
Test Accuracy: 0.7698675496688742


In [40]:
svm_sgd = SGDClassifier(loss='hinge', penalty='l2', max_iter=5, random_state=42)
svm_sgd.fit(tv_train_features, train_label_names)
svmsgd_tfidf_cv_scores = cross_val_score(svm_sgd, tv_train_features, train_label_names, cv=5)
svmsgd_tfidf_cv_mean_score = np.mean(svmsgd_tfidf_cv_scores)
print('CV Accuracy (5-fold):', svmsgd_tfidf_cv_scores)
print('Mean CV Accuracy:', svmsgd_tfidf_cv_mean_score)
svmsgd_tfidf_test_score = svm_sgd.score(tv_test_features, test_label_names)
print('Test Accuracy:', svmsgd_tfidf_test_score)

CV Accuracy (5-fold): [0.75866286 0.73613377 0.7687602  0.77243067 0.75163132]
Mean CV Accuracy: 0.7575237632249754
Test Accuracy: 0.7683774834437086


In [41]:
rfc = RandomForestClassifier(n_estimators=10, random_state=42)
rfc.fit(tv_train_features, train_label_names)
rfc_tfidf_cv_scores = cross_val_score(rfc, tv_train_features, train_label_names, cv=5)
rfc_tfidf_cv_mean_score = np.mean(rfc_tfidf_cv_scores)
print('CV Accuracy (5-fold):', rfc_tfidf_cv_scores)
print('Mean CV Accuracy:', rfc_tfidf_cv_mean_score)
rfc_tfidf_test_score = rfc.score(tv_test_features, test_label_names)
print('Test Accuracy:', rfc_tfidf_test_score)

CV Accuracy (5-fold): [0.52629433 0.52528548 0.53548124 0.53548124 0.53711256]
Mean CV Accuracy: 0.5319309710984119
Test Accuracy: 0.5183774834437086


In [42]:
gbc = GradientBoostingClassifier(n_estimators=10, random_state=42)
gbc.fit(tv_train_features, train_label_names)
gbc_tfidf_cv_scores = cross_val_score(gbc, tv_train_features, train_label_names, cv=5)
gbc_tfidf_cv_mean_score = np.mean(gbc_tfidf_cv_scores)
print('CV Accuracy (5-fold):', gbc_tfidf_cv_scores)
print('Mean CV Accuracy:', gbc_tfidf_cv_mean_score)
gbc_tfidf_test_score = gbc.score(tv_test_features, test_label_names)
print('Test Accuracy:', gbc_tfidf_test_score)

CV Accuracy (5-fold): [0.54912352 0.53344209 0.56606852 0.5725938  0.55628059]
Mean CV Accuracy: 0.5555017028122172
Test Accuracy: 0.5498344370860927


In [47]:
#Ergebnisübersicht

pd.DataFrame([['Naive Bayes', mnb_bow_cv_mean_score, mnb_bow_test_score, 
               mnb_tfidf_cv_mean_score, mnb_tfidf_test_score],
              ['Logistic Regression', lr_bow_cv_mean_score, lr_bow_test_score, 
               lr_tfidf_cv_mean_score, lr_tfidf_test_score],
              ['Linear SVM', svm_bow_cv_mean_score, svm_bow_test_score, 
               svm_tfidf_cv_mean_score, svm_tfidf_test_score],
              ['Linear SVM (SGD)', svmsgd_bow_cv_mean_score, svmsgd_bow_test_score, 
               svmsgd_tfidf_cv_mean_score, svmsgd_tfidf_test_score],
              ['Random Forest', rfc_bow_cv_mean_score, rfc_bow_test_score, 
               rfc_tfidf_cv_mean_score, rfc_tfidf_test_score],
              ['Gradient Boosted Machines', gbc_bow_cv_mean_score, gbc_bow_test_score, 
               gbc_tfidf_cv_mean_score, gbc_tfidf_test_score]],
             columns=['Model', 'CV Score (TF)', 'Test Score (TF)', 'CV Score (TF-IDF)', 'Test Score (TF-IDF)'],
             ).T

,0,1,2,3,4,5
Model,Naive Bayes,Logistic Regression,Linear SVM,Linear SVM (SGD),Random Forest,Gradient Boosted Machines
CV Score (TF),0.669031,0.681021,0.640079,0.638773,0.515129,0.551342
Test Score (TF),0.681954,0.692053,0.656954,0.651987,0.526821,0.555132
CV Score (TF-IDF),0.708425,0.740315,0.754017,0.757524,0.531931,0.555502
Test Score (TF-IDF),0.710099,0.752152,0.769868,0.768377,0.518377,0.549834


In [48]:
#hier startet das Hyperparameter-Tuning

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

mnb_pipeline = Pipeline([('tfidf', TfidfVectorizer()),
                        ('mnb', MultinomialNB())
                       ])

#die Parameter, die durchprobiert werden sollen: 1/2-gramme und alpha-smoothing
param_grid = {'tfidf__ngram_range': [(1, 1), (1, 2)],
              'mnb__alpha': [1e-5, 1e-4, 1e-2, 1e-1, 1]
}

gs_mnb = GridSearchCV(mnb_pipeline, param_grid, cv=5, verbose=2)
gs_mnb = gs_mnb.fit(train_corpus, train_label_names)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END ........mnb__alpha=1e-05, tfidf__ngram_range=(1, 1); total time=   0.4s
[CV] END ........mnb__alpha=1e-05, tfidf__ngram_range=(1, 1); total time=   0.4s
[CV] END ........mnb__alpha=1e-05, tfidf__ngram_range=(1, 1); total time=   0.5s
[CV] END ........mnb__alpha=1e-05, tfidf__ngram_range=(1, 1); total time=   0.4s
[CV] END ........mnb__alpha=1e-05, tfidf__ngram_range=(1, 1); total time=   0.4s
[CV] END ........mnb__alpha=1e-05, tfidf__ngram_range=(1, 2); total time=   2.1s
[CV] END ........mnb__alpha=1e-05, tfidf__ngram_range=(1, 2); total time=   2.2s
[CV] END ........mnb__alpha=1e-05, tfidf__ngram_range=(1, 2); total time=   2.2s
[CV] END ........mnb__alpha=1e-05, tfidf__ngram_range=(1, 2); total time=   2.2s
[CV] END ........mnb__alpha=1e-05, tfidf__ngram_range=(1, 2); total time=   2.2s
[CV] END .......mnb__alpha=0.0001, tfidf__ngram_range=(1, 1); total time=   0.4s
[CV] END .......mnb__alpha=0.0001, tfidf__ngram_

In [49]:
#beste Parameter?
gs_mnb.best_estimator_.get_params()

{'memory': None,
 'steps': [('tfidf', TfidfVectorizer(ngram_range=(1, 2))),
  ('mnb', MultinomialNB(alpha=0.01))],
 'verbose': False,
 'tfidf': TfidfVectorizer(ngram_range=(1, 2)),
 'mnb': MultinomialNB(alpha=0.01),
 'tfidf__analyzer': 'word',
 'tfidf__binary': False,
 'tfidf__decode_error': 'strict',
 'tfidf__dtype': numpy.float64,
 'tfidf__encoding': 'utf-8',
 'tfidf__input': 'content',
 'tfidf__lowercase': True,
 'tfidf__max_df': 1.0,
 'tfidf__max_features': None,
 'tfidf__min_df': 1,
 'tfidf__ngram_range': (1, 2),
 'tfidf__norm': 'l2',
 'tfidf__preprocessor': None,
 'tfidf__smooth_idf': True,
 'tfidf__stop_words': None,
 'tfidf__strip_accents': None,
 'tfidf__sublinear_tf': False,
 'tfidf__token_pattern': '(?u)\\b\\w\\w+\\b',
 'tfidf__tokenizer': None,
 'tfidf__use_idf': True,
 'tfidf__vocabulary': None,
 'mnb__alpha': 0.01,
 'mnb__class_prior': None,
 'mnb__fit_prior': True,
 'mnb__force_alpha': 'warn'}

In [50]:
#Rangfolge

cv_results = gs_mnb.cv_results_
results_df = pd.DataFrame({'rank': cv_results['rank_test_score'],
                           'params': cv_results['params'], 
                           'cv score (mean)': cv_results['mean_test_score'], 
                           'cv score (std)': cv_results['std_test_score']} 
              )
results_df = results_df.sort_values(by=['rank'], ascending=True)
pd.set_option('display.max_colwidth', 100)
results_df

,rank,params,cv score (mean),cv score (std)
5,1,"{'mnb__alpha': 0.01, 'tfidf__ngram_range': (1, 2)}",0.767718,0.007810
4,2,"{'mnb__alpha': 0.01, 'tfidf__ngram_range': (1, 1)}",0.764945,0.010373
7,3,"{'mnb__alpha': 0.1, 'tfidf__ngram_range': (1, 2)}",0.752303,0.009830
6,4,"{'mnb__alpha': 0.1, 'tfidf__ngram_range': (1, 1)}",0.752222,0.011394
3,5,"{'mnb__alpha': 0.0001, 'tfidf__ngram_range': (1, 2)}",0.751162,0.007427
1,6,"{'mnb__alpha': 1e-05, 'tfidf__ngram_range': (1, 2)}",0.741620,0.008421
2,7,"{'mnb__alpha': 0.0001, 'tfidf__ngram_range': (1, 1)}",0.737786,0.011859
0,8,"{'mnb__alpha': 1e-05, 'tfidf__ngram_range': (1, 1)}",0.725960,0.013517
8,9,"{'mnb__alpha': 1, 'tfidf__ngram_range': (1, 1)}",0.710219,0.009885
9,10,"{'mnb__alpha': 1, 'tfidf__ngram_range': (1, 2)}",0.702798,0.010097


# Pickle trained model so that we do not have to retrain it again

In [69]:
import pickle
with open("./trained_models/gs_mnb.model", "wb") as f:
    pickle.dump(gs_mnb, f)

In [51]:
#und wie gut geht es für die Testdaten?
best_mnb_test_score = gs_mnb.score(test_corpus, test_label_names)
print('Test Accuracy :', best_mnb_test_score)

Test Accuracy : 0.7809602649006623


In [52]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

lr_pipeline = Pipeline([('tfidf', TfidfVectorizer()),
                        ('lr', LogisticRegression(penalty='l2', max_iter=100, random_state=42))
                       ])

#C-Wert steuert Regularisierung = Bestrafung von over-fitting
# komplexe Funktion wird weniger gut bewertet als einfache
param_grid = {'tfidf__ngram_range': [(1, 1), (1, 2)],
              'lr__C': [1, 5, 10]
}

gs_lr = GridSearchCV(lr_pipeline, param_grid, cv=5, verbose=2)
gs_lr = gs_lr.fit(train_corpus, train_label_names)

In [150]:
gs_lr.best_estimator_

Pipeline(steps=[('tfidf', TfidfVectorizer(ngram_range=(1, 2))),
                ('lr', LogisticRegression(C=10, random_state=42))])

In [151]:
best_lr_test_score = gs_lr.score(test_corpus, test_label_names)
print('Test Accuracy :', best_lr_test_score)

Test Accuracy : 0.769205298013245


In [152]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

svm_pipeline = Pipeline([('tfidf', TfidfVectorizer()),
                        ('svm', LinearSVC(random_state=42))
                       ])

#C-Wert - siehe oben
param_grid = {'tfidf__ngram_range': [(1, 1), (1, 2)],
              'svm__C': [0.01, 0.1, 1, 5]
}

gs_svm = GridSearchCV(svm_pipeline, param_grid, cv=5, verbose=2)
gs_svm = gs_svm.fit(train_corpus, train_label_names)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END .............svm__C=0.01, tfidf__ngram_range=(1, 1); total time=   3.5s
[CV] END .............svm__C=0.01, tfidf__ngram_range=(1, 1); total time=   3.2s
[CV] END .............svm__C=0.01, tfidf__ngram_range=(1, 1); total time=   3.2s
[CV] END .............svm__C=0.01, tfidf__ngram_range=(1, 1); total time=   3.2s
[CV] END .............svm__C=0.01, tfidf__ngram_range=(1, 1); total time=   3.3s
[CV] END .............svm__C=0.01, tfidf__ngram_range=(1, 2); total time=  11.0s
[CV] END .............svm__C=0.01, tfidf__ngram_range=(1, 2); total time=  10.8s
[CV] END .............svm__C=0.01, tfidf__ngram_range=(1, 2); total time=  10.9s
[CV] END .............svm__C=0.01, tfidf__ngram_range=(1, 2); total time=  11.0s
[CV] END .............svm__C=0.01, tfidf__ngram_range=(1, 2); total time=  11.2s
[CV] END ..............svm__C=0.1, tfidf__ngram_range=(1, 1); total time=   3.2s
[CV] END ..............svm__C=0.1, tfidf__ngram_r

In [153]:
gs_svm.best_estimator_.get_params()

{'memory': None,
 'steps': [('tfidf', TfidfVectorizer(ngram_range=(1, 2))),
  ('svm', LinearSVC(C=5, random_state=42))],
 'verbose': False,
 'tfidf': TfidfVectorizer(ngram_range=(1, 2)),
 'svm': LinearSVC(C=5, random_state=42),
 'tfidf__analyzer': 'word',
 'tfidf__binary': False,
 'tfidf__decode_error': 'strict',
 'tfidf__dtype': numpy.float64,
 'tfidf__encoding': 'utf-8',
 'tfidf__input': 'content',
 'tfidf__lowercase': True,
 'tfidf__max_df': 1.0,
 'tfidf__max_features': None,
 'tfidf__min_df': 1,
 'tfidf__ngram_range': (1, 2),
 'tfidf__norm': 'l2',
 'tfidf__preprocessor': None,
 'tfidf__smooth_idf': True,
 'tfidf__stop_words': None,
 'tfidf__strip_accents': None,
 'tfidf__sublinear_tf': False,
 'tfidf__token_pattern': '(?u)\\b\\w\\w+\\b',
 'tfidf__tokenizer': None,
 'tfidf__use_idf': True,
 'tfidf__vocabulary': None,
 'svm__C': 5,
 'svm__class_weight': None,
 'svm__dual': 'warn',
 'svm__fit_intercept': True,
 'svm__intercept_scaling': 1,
 'svm__loss': 'squared_hinge',
 'svm__max_ite

In [154]:
best_svm_test_score = gs_svm.score(test_corpus, test_label_names)
print('Test Accuracy :', best_svm_test_score)

Test Accuracy : 0.7832781456953642


In [155]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

sgd_pipeline = Pipeline([('tfidf', TfidfVectorizer()),
                        ('sgd', SGDClassifier(random_state=42))
                       ])

#alpha-Wert: Regularisierungsfaktor
param_grid = {'tfidf__ngram_range': [(1, 1), (1, 2)],
              'sgd__alpha': [1e-7, 1e-6, 1e-5, 1e-4]
}

gs_sgd = GridSearchCV(sgd_pipeline, param_grid, cv=5, verbose=2)
gs_sgd = gs_sgd.fit(train_corpus, train_label_names)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END ........sgd__alpha=1e-07, tfidf__ngram_range=(1, 1); total time=   3.2s
[CV] END ........sgd__alpha=1e-07, tfidf__ngram_range=(1, 1); total time=   2.9s
[CV] END ........sgd__alpha=1e-07, tfidf__ngram_range=(1, 1); total time=   3.1s
[CV] END ........sgd__alpha=1e-07, tfidf__ngram_range=(1, 1); total time=   3.0s
[CV] END ........sgd__alpha=1e-07, tfidf__ngram_range=(1, 1); total time=   2.9s
[CV] END ........sgd__alpha=1e-07, tfidf__ngram_range=(1, 2); total time=   9.4s
[CV] END ........sgd__alpha=1e-07, tfidf__ngram_range=(1, 2); total time=   9.9s
[CV] END ........sgd__alpha=1e-07, tfidf__ngram_range=(1, 2); total time=  10.3s
[CV] END ........sgd__alpha=1e-07, tfidf__ngram_range=(1, 2); total time=  12.5s
[CV] END ........sgd__alpha=1e-07, tfidf__ngram_range=(1, 2); total time=  10.5s
[CV] END ........sgd__alpha=1e-06, tfidf__ngram_range=(1, 1); total time=   2.7s
[CV] END ........sgd__alpha=1e-06, tfidf__ngram_r

In [156]:
gs_sgd.best_estimator_.get_params()

{'memory': None,
 'steps': [('tfidf', TfidfVectorizer(ngram_range=(1, 2))),
  ('sgd', SGDClassifier(random_state=42))],
 'verbose': False,
 'tfidf': TfidfVectorizer(ngram_range=(1, 2)),
 'sgd': SGDClassifier(random_state=42),
 'tfidf__analyzer': 'word',
 'tfidf__binary': False,
 'tfidf__decode_error': 'strict',
 'tfidf__dtype': numpy.float64,
 'tfidf__encoding': 'utf-8',
 'tfidf__input': 'content',
 'tfidf__lowercase': True,
 'tfidf__max_df': 1.0,
 'tfidf__max_features': None,
 'tfidf__min_df': 1,
 'tfidf__ngram_range': (1, 2),
 'tfidf__norm': 'l2',
 'tfidf__preprocessor': None,
 'tfidf__smooth_idf': True,
 'tfidf__stop_words': None,
 'tfidf__strip_accents': None,
 'tfidf__sublinear_tf': False,
 'tfidf__token_pattern': '(?u)\\b\\w\\w+\\b',
 'tfidf__tokenizer': None,
 'tfidf__use_idf': True,
 'tfidf__vocabulary': None,
 'sgd__alpha': 0.0001,
 'sgd__average': False,
 'sgd__class_weight': None,
 'sgd__early_stopping': False,
 'sgd__epsilon': 0.1,
 'sgd__eta0': 0.0,
 'sgd__fit_intercept': 

In [157]:
best_sgd_test_score = gs_sgd.score(test_corpus, test_label_names)
print('Test Accuracy :', best_sgd_test_score)

Test Accuracy : 0.7768211920529802


In [ ]:
import model_evaluation_utils_hr as meu
import importlib
importlib.reload(meu)

In [56]:
#Analyse der Ergebnisse für MNB
# Auswahl aufgrund guter Ergebnisse und einfaches Handling

mnb_predictions = gs_mnb.predict(test_corpus)
unique_classes = list(set(test_label_names))
meu.get_metrics(true_labels=test_label_names, predicted_labels=mnb_predictions)

Accuracy: 0.781
Precision: 0.7904
Recall: 0.781
F1 Score: 0.7775


In [57]:
#Ergebnisse nach Newsgruppe

meu.display_classification_report(true_labels=test_label_names, 
                                  predicted_labels=mnb_predictions, classes=unique_classes)

                          precision    recall  f1-score   support

               sci.space       0.85      0.80      0.82       335
      rec.sport.baseball       0.93      0.88      0.91       310
        rec.sport.hockey       0.90      0.91      0.91       318
 comp.os.ms-windows.misc       0.72      0.70      0.71       304
      talk.politics.misc       0.72      0.70      0.71       260
               rec.autos       0.82      0.79      0.81       303
   comp.sys.mac.hardware       0.79      0.77      0.78       296
   talk.politics.mideast       0.84      0.84      0.84       321
comp.sys.ibm.pc.hardware       0.73      0.71      0.72       343
      talk.politics.guns       0.67      0.81      0.74       296
  soc.religion.christian       0.60      0.92      0.72       318
            misc.forsale       0.86      0.75      0.80       322
         rec.motorcycles       0.84      0.81      0.82       311
             alt.atheism       0.83      0.56      0.67       290
         

In [58]:
#Übersicht Label

label_data_map = {v:k for k, v in data_labels_map.items()}
label_map_df = pd.DataFrame(list(label_data_map.items()), columns=['Label Name', 'Label Number'])
label_map_df

,Label Name,Label Number
0,alt.atheism,0
1,comp.graphics,1
2,comp.os.ms-windows.misc,2
3,comp.sys.ibm.pc.hardware,3
4,comp.sys.mac.hardware,4
5,comp.windows.x,5
6,misc.forsale,6
7,rec.autos,7
8,rec.motorcycles,8
9,rec.sport.baseball,9


In [59]:
#Konfusionsmatrix
unique_class_nums = label_map_df['Label Number'].values
mnb_prediction_class_nums = [label_data_map[item] for item in mnb_predictions]
meu.display_confusion_matrix_pretty(true_labels=test_label_nums, 
                                   predicted_labels=mnb_prediction_class_nums, classes=unique_class_nums)

Predicted:                                                         \
                   0    1    2    3    4    5    6    7    8    9    10   11   
Actual: 0         163    3    2    1    0    1    0    3    3    0    4    6   
        1           2  224    8   12    4   17    2    0    1    2    1    8   
        2           1   19  212   25   13   19    0    0    0    0    0    5   
        3           1   15   27  243   22    4    7    3    0    0    0    5   
        4           0   14    9   14  227    3    6    2    1    1    0    6   
        5           1   21   12    4    2  257    1    0    0    0    0    4   
        6           0    3    5   19    6    0  243   12    3    0    2    7   
        7           0    3    3    0    0    0    8  240   18    1    0    3   
        8           1    4    0    1    0    0    4   13  252    4    4    1   
        9           2    4    0    1    0    1    0    1    1  274    9    3   
        10          1    1    2    1    0    2    0    1    3    5  290    1   
        11          0    2    1    0    1    1    0    0    1    0    0  260   
        12          1    8    5    8    9    0    3   11    3    0    1    9   
        13          1    4    1    1    1    3    2    0    2    0    0    1   
        14          0    8    1    1    1    3    2    4    3    2    2    5   
        15          4    2    2    0    2    1    1    0    0    1    0    0   
        16          0    0    2    0    0    2    2    0    4    0    4    9   
        17          0    0    1    1    0    0    0    1    3    2    1    5   
        18          2    1    1    0    1    0    0    1    1    2    2    4   
        19         17    0    1    0    0    1    1    1    2    1    1    1   

                                                   
             12   13   14   15   16   17   18  19  
Actual: 0     2    1    3   56   10   15    7  10  
        1     1    2    5    0    2    0    1   0  
        2     4    1    2    1    0    0    2   0  
        3    15    0    1    0    0    0    0   0  
        4     8    0    2    1    0    0    2   0  
        5     4    0    1    3    0    0    1   0  
        6     8    2    5    3    1    2    1   0  
        7     9    1    0    4    9    0    4   0  
        8     4    4    4    7    3    1    4   0  
        9     1    1    0    6    5    0    1   0  
        10    0    3    2    1    0    2    3   0  
        11    2    0    1    4    8    1    5   0  
        12  244    3    3    1    2    0    0   0  
        13    7  279    5    7    0    1    3   1  
        14    7    6  268    9    3    4    6   0  
        15    0    2    3  293    2    1    1   3  
        16    1    0    3    3  241    6   19   0  
        17    0    3    3   12   11  271    7   0  
        18    0    4    3    8   36   11  183   0  
        19    0    3    2   72   26    6    5  53

In [60]:
#Konfusionsmatrix mit Labeln
unique_classes = label_map_df['Label Name'].values
meu.display_confusion_matrix_pretty(true_labels=test_label_names, 
                                    predicted_labels=mnb_predictions, classes=unique_classes)

Predicted:                \
                                 alt.atheism comp.graphics   
Actual: alt.atheism                      163             3   
        comp.graphics                      2           224   
        comp.os.ms-windows.misc            1            19   
        comp.sys.ibm.pc.hardware           1            15   
        comp.sys.mac.hardware              0            14   
        comp.windows.x                     1            21   
        misc.forsale                       0             3   
        rec.autos                          0             3   
        rec.motorcycles                    1             4   
        rec.sport.baseball                 2             4   
        rec.sport.hockey                   1             1   
        sci.crypt                          0             2   
        sci.electronics                    1             8   
        sci.med                            1             4   
        sci.space                          0             8   
        soc.religion.christian             4             2   
        talk.politics.guns                 0             0   
        talk.politics.mideast              0             0   
        talk.politics.misc                 2             1   
        talk.religion.misc                17             0   

                                                          \
                                 comp.os.ms-windows.misc   
Actual: alt.atheism                                    2   
        comp.graphics                                  8   
        comp.os.ms-windows.misc                      212   
        comp.sys.ibm.pc.hardware                      27   
        comp.sys.mac.hardware                          9   
        comp.windows.x                                12   
        misc.forsale                                   5   
        rec.autos                                      3   
        rec.motorcycles                                0   
        rec.sport.baseball                             0   
        rec.sport.hockey                               2   
        sci.crypt                                      1   
        sci.electronics                                5   
        sci.med                                        1   
        sci.space                                      1   
        soc.religion.christian                         2   
        talk.politics.guns                             2   
        talk.politics.mideast                          1   
        talk.politics.misc                             1   
        talk.religion.misc                             1   

                                                           \
                                 comp.sys.ibm.pc.hardware   
Actual: alt.atheism                                     1   
        comp.graphics                                  12   
        comp.os.ms-windows.misc                        25   
        comp.sys.ibm.pc.hardware                      243   
        comp.sys.mac.hardware                          14   
        comp.windows.x                                  4   
        misc.forsale                                   19   
        rec.autos                                       0   
        rec.motorcycles                                 1   
        rec.sport.baseball                              1   
        rec.sport.hockey                                1   
        sci.crypt                                       0   
        sci.electronics                                 8   
        sci.med                                         1   
        sci.space                                       1   
        soc.religion.christian                          0   
        talk.politics.guns                              0   
        talk.politics.mideast                           1   
        talk.politics.misc                              0   
        talk.religion.misc                              0   

     

In [61]:
#wo sind die Ausreisser?

label_map_df[label_map_df['Label Number'].isin([0, 15, 19])]

,Label Name,Label Number
0,alt.atheism,0
15,soc.religion.christian,15
19,talk.religion.misc,19


In [62]:
train_idx, test_idx = train_test_split(np.array(range(len(data_df['Article']))), test_size=0.33, random_state=42)
test_idx

array([ 4097,  8528,  7621, ...,  4772,  7800, 14579])

In [63]:
#einige Beispiele

predict_probas = gs_mnb.predict_proba(test_corpus).max(axis=1)
test_df = data_df.iloc[test_idx]
test_df['Predicted Name'] = mnb_predictions
test_df['Predicted Confidence'] = predict_probas
test_df.head()

,Article,Clean Article,Target Label,Target Name,Predicted Name,Predicted Confidence
4097,one way to get the system going with one floppy drive and one hard\ndisk on a 63 watt power supp...,one way get system go one floppy drive one hard disk watt power supply first disconnect power fl...,3,comp.sys.ibm.pc.hardware,comp.sys.ibm.pc.hardware,0.612672
8528,"\n\n\n\nI've already written a 5000 char commentary (from my MCI MAIL account, so I\ncan't be ac...",already write char commentary mci mail account not accuse hacker,11,sci.crypt,sci.crypt,0.939573
7621,\n\n\tWhat alternative would you suggest be taken to safeguard the\nlives of Israeli citizens?\n...,alternative would suggest take safeguard life israeli citizen adam adam shostack adamdas harvard...,17,talk.politics.mideast,talk.politics.mideast,0.999790
4754,I have already purchased 72-pin SIMMs for a Quadra 800 from\nMemory Direct (on March 9). How ca...,already purchase pin simm quadra memory direct march tell whether not composite simm rule memory...,4,comp.sys.mac.hardware,comp.sys.mac.hardware,0.984552
15905,I'd say the subject line about covers it. I need a MC68881 Floating point\n Co-processor for ...,would say subject line cover need mc floating point co processor sun system please reply via ema...,6,misc.forsale,misc.forsale,0.587902


In [64]:
#falsch einsortierte Dokumente

pd.set_option('display.max_colwidth', 200)
res_df = (test_df[(test_df['Target Name'] == 'talk.religion.misc') & (test_df['Predicted Name'] == 'soc.religion.christian')]
       .sort_values(by=['Predicted Confidence'], ascending=False).head(5))
res_df

,Article,Clean Article,Target Label,Target Name,Predicted Name,Predicted Confidence
14514,"iank@microsoft.com (Ian Kennedy) writes...\n\n\nMore along the lines of Hebrews 12:25-29, I reckon...\n\n\tSee that you refuse not him that speaks. For if they\n\tescaped not who refused him that ...",iankmicrosoft com ian kennedy write along line hebrews reckon see refuse not speak escape not refuse spake earth much shall not escape turn away speak heaven whose voice shake earth promise say ye...,19,talk.religion.misc,soc.religion.christian,0.999978
4367,:\n (lots of stuff about the Nicene Creed deleted which can be read in the\n original basenote. I will also leave it up to other LDS netters to\n take Mr. Weiss to task on using Mormon Doctrine...,lot stuff nicene creed delete read original basenote also leave lds netter take mr weiss task use mormon doctrine declare difinitive word lds church teach doctrine hopefully lds netter amiable exp...,19,talk.religion.misc,soc.religion.christian,0.998596
4186,"\nRick, I think we can safely say, 1) Robert is not the only person\nwho understands the Bible, and 2), the leadership of the LDS church\nhistoricly never has. Let's consider some ""personal inter...",rick think safely say robert not person understand bible leadership lds church historicly never let consider personal interpretation see much trust put orthodox mormonism could never confuse ortho...,19,talk.religion.misc,soc.religion.christian,0.998148
8968,"\nZoroaster is far older than Daniel. If anything, one could claim that,\nin a sense, Daniel is a descendant of Zoroaster; as Daniel, though being\nHebrew, has assimilated into Zoroastrianism and ...",zoroaster far old daniel anything one could claim sense daniel descendant zoroaster daniel though hebrew assimilate zoroastrianism successfully introduce religion tanakh judaism however majority b...,19,talk.religion.misc,soc.religion.christian,0.998124
16679,"\nJesus did not say that he was the fulfillment of the Law, and, unless\nI'm mistaken, heaven and earth have not yet passed away. Am I mistaken?\nAnd, even assuming that one can just gloss over th...",jesus not say fulfillment law unless mistaken heaven earth not yet pass away mistaken even assume one gloss portion word jesus really think accomplish not jesus say jew annul v say jesus record wo...,19,talk.religion.misc,soc.religion.christian,0.997866


In [65]:
#weitere Falschsortierungen (andere Gruppe)

pd.set_option('display.max_colwidth', 200)
res_df = (test_df[(test_df['Target Name'] == 'talk.religion.misc') & (test_df['Predicted Name'] == 'alt.atheism')]
       .sort_values(by=['Predicted Confidence'], ascending=False).head(5))
res_df

,Article,Clean Article,Target Label,Target Name,Predicted Name,Predicted Confidence
2467,"Why is the NT tossed out as info on Jesus. I realize it is normally tossed\nout because it contains miracles, but what are the other reasons?\n\nMAC\n--\n*****************************************...",nt toss info jesus realize normally toss contain miracle reason mac michael cobb not raise taxis middle university illinois class pay program champaign urbana bill clinton rd debate cobbalexia lis...,19,talk.religion.misc,alt.atheism,0.999983
3429,"\n\n\nIs there any difference in saying \n\n""Absolute Truth exists, but some people think its a lie""\n\nand\n\n""Truth is relative"" ?\n\nI think there is: in both examples, the first statement is ...",difference say absolute truth exist people think lie truth relative think example first statement fundamental disagreement least two people second statement agree upon put another way someone say ...,19,talk.religion.misc,alt.atheism,0.857900
1675,"\n[...stuff deleted...]\n\nAndy-- I think we do agree, given your clarification of how we were \neach using the terms fact and theory. I'll only add that I\nthink perhaps I feel more strongly abo...",stuff delete andy think agree give clarification use term fact theory add think perhaps feel strongly separate though usage quite valid add falsification rejection not way reduce current usefulnes...,19,talk.religion.misc,alt.atheism,0.841454
12384,"\nAh, you taking everything as literal quotation. No wonder you're confused.\n\nFirst, can I ask that we decide on a definition of ""objective""?\n\n\nAnd?\n\n\nI'd guess that it might be.\n\n\nIt ...",ah take everything literal quotation no wonder confused first ask decide definition objective would guess might may case people unable evaluate complex moral issue rather leave behave immorally mi...,19,talk.religion.misc,alt.atheism,0.805937
14774,[ deleted ]\n[ deleted ]\n\n Evolution is both fact and theory. The THEORY of evolution represents the\nscientific attempt to explain the FACT of evolution. The theory of evolution\ndoes not pr...,delete delete evolution fact theory theory evolution represent scientific attempt explain fact evolution theory evolution not provide fact explain fact safely assume scientific theory neither prov...,19,talk.religion.misc,alt.atheism,0.782795
